Reading Data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
from glob import iglob
import anndata
import os
import sklearn
from sklearn.linear_model import LogisticRegression
import pickle
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
np.random.seed(0)
sc.settings.verbosity = 3
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sc.settings.set_figure_params(dpi = 100)

In [2]:
save_path = '/lustre/scratch117/cellgen/team292/ab55/'

In [3]:
# load data
adata = anndata.read_h5ad(save_path+"N3-pbmc-final-clustering.h5ad")
adata

AnnData object with n_obs × n_vars = 97499 × 22572
    obs: 'souporcell', 'demultiplexed', 'sample_names', 'log2p1_count', 'percent_mito', 'n_genes', 'batch', 'FolderName', 'Lane', 'Sort', 'Type', 'Donor Id', 'Age', 'Sex', 'Race', 'Ethnicity', 'BMI', 'Pre-existing heart disease', 'Pre-existing lung disease', 'Pre-existing kidney disease', 'Pre-existing diabetes', 'Pre-existing Hypertension', 'Pre-existing immunocompromised condition', 'Smoking', 'SARS-CoV-2 PCR', 'SARS-CoV-2 Ab', 'Symptomatic', 'Admitted to hospital', 'Highest level of respiratory support', 'Vasoactive agents required during hospitalization', '28-day death', 'scrublet_pred', 'scrublet_local_pred', 'scrublet_score', 'scrublet_cluster_score', 'filtered_cells', 'S_score', 'G2M_score', 'phase', 'leiden_sampl_cc', 'leidenres2_sampl_cc', 'DonorSubset', 'leiden_scvi_subset_cc', 'leidenres2_scvi_subset_cc', 'leiden_tvi_subset_cc', 'leidenres2_tvi_subset_cc', 'louvain', 'Celltype_Predictions', 'Prediction_Probabilities', 'Final

In [4]:
# load reference dataset (Stephenson, E. et al, Nature Medicine, 2021)
adata_ref = anndata.read_h5ad(save_path + "haniffa21_processed_raw.h5ad")
adata_ref

AnnData object with n_obs × n_vars = 691683 × 31279
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'Resample', 'Collection_Day', 'Sex', 'Age', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'initial_clustering', 'full_clustering'
    var: 'feature_types'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

Processing Data

In [5]:
adata = anndata.AnnData(X = adata.raw.X, obs = adata.obs, var = adata.raw.var, obsm = adata.obsm).copy()
sc.pp.filter_genes(adata, min_cells = 3)
sc.pp.normalize_per_cell(adata, counts_per_cell_after = 1e4)
sc.pp.log1p(adata)
adata

filtered out 11187 genes that are detected in less than 3 cells
/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
normalizing by total count per cell
    finished (0:00:01): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


AnnData object with n_obs × n_vars = 97499 × 22572
    obs: 'souporcell', 'demultiplexed', 'sample_names', 'log2p1_count', 'percent_mito', 'n_genes', 'batch', 'FolderName', 'Lane', 'Sort', 'Type', 'Donor Id', 'Age', 'Sex', 'Race', 'Ethnicity', 'BMI', 'Pre-existing heart disease', 'Pre-existing lung disease', 'Pre-existing kidney disease', 'Pre-existing diabetes', 'Pre-existing Hypertension', 'Pre-existing immunocompromised condition', 'Smoking', 'SARS-CoV-2 PCR', 'SARS-CoV-2 Ab', 'Symptomatic', 'Admitted to hospital', 'Highest level of respiratory support', 'Vasoactive agents required during hospitalization', '28-day death', 'scrublet_pred', 'scrublet_local_pred', 'scrublet_score', 'scrublet_cluster_score', 'filtered_cells', 'S_score', 'G2M_score', 'phase', 'leiden_sampl_cc', 'leidenres2_sampl_cc', 'DonorSubset', 'leiden_scvi_subset_cc', 'leidenres2_scvi_subset_cc', 'leiden_tvi_subset_cc', 'leidenres2_tvi_subset_cc', 'louvain', 'Celltype_Predictions', 'Prediction_Probabilities', 'Final

In [6]:
# using Ensembl Biomart list to filter genes on the Y chromosome
Y_chr_table = pd.read_csv('/home/jovyan/COVID/NB5_DE Analysis/Y_chr_genes.txt')
Y_chr = list(Y_chr_table['Gene name'])
all_genes = adata.var.index
genes_to_remove = list(set(all_genes) & set(Y_chr))
all_genes = [ele for ele in all_genes if ele not in genes_to_remove]
adata = adata[: , all_genes]
adata

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


View of AnnData object with n_obs × n_vars = 97499 × 22553
    obs: 'souporcell', 'demultiplexed', 'sample_names', 'log2p1_count', 'percent_mito', 'n_genes', 'batch', 'FolderName', 'Lane', 'Sort', 'Type', 'Donor Id', 'Age', 'Sex', 'Race', 'Ethnicity', 'BMI', 'Pre-existing heart disease', 'Pre-existing lung disease', 'Pre-existing kidney disease', 'Pre-existing diabetes', 'Pre-existing Hypertension', 'Pre-existing immunocompromised condition', 'Smoking', 'SARS-CoV-2 PCR', 'SARS-CoV-2 Ab', 'Symptomatic', 'Admitted to hospital', 'Highest level of respiratory support', 'Vasoactive agents required during hospitalization', '28-day death', 'scrublet_pred', 'scrublet_local_pred', 'scrublet_score', 'scrublet_cluster_score', 'filtered_cells', 'S_score', 'G2M_score', 'phase', 'leiden_sampl_cc', 'leidenres2_sampl_cc', 'DonorSubset', 'leiden_scvi_subset_cc', 'leidenres2_scvi_subset_cc', 'leiden_tvi_subset_cc', 'leidenres2_tvi_subset_cc', 'louvain', 'Celltype_Predictions', 'Prediction_Probabilities'

In [7]:
# list of surface proteins profiled
protein_list = np.array(adata.var_names)[adata.var["feature_types"] ==  "Antibody Capture"].tolist()
len(protein_list)

192

In [8]:
# add donor condition information to be used for the different comparisons
donor2condition = {
     'CV19-I-O-2': 'MS',
     'CV19-I-O-3': 'MS',
     'CV19-I-O-73': 'MS',
     'CV19-I-O-34': 'PS',
     'CV19-I-O-59': 'PS',
     'CV19-I-O-71': 'PS',
     'CV19-I-O-74': 'PS',
     'CV19-I-O-4': 'RA',
     'CV19-I-O-39': 'RA',
     'CV19-I-O-40': 'RA',
     'CV19-I-O-45': 'RA',
     'CV19-I-O-51': 'RA',
     'CV19-I-O-14': 'Control',
     'CV19-I-O-16': 'Control',
     'CV19-I-O-17': 'Control',
     'CV19-I-O-18': 'Control',
     'CV19-I-O-19': 'Control',
     'CV19-I-O-20': 'Control',
     'CV19-I-O-21': 'Control',
     'CV19-I-O-22': 'Control',
     'CV19-I-O-23': 'Control',
     'CV19-I-O-24': 'Control'}
adata.obs['Immune_condition'] = adata.obs['demultiplexed'].map(donor2condition).astype('category')
adata.obs['celltype_condition'] = [i + '_' + j for i,j in zip(adata.obs['Final_Celltype'], adata.obs['Immune_condition'])]

donor2condition = {
     'CV19-I-O-2': 'Other',
     'CV19-I-O-3': 'Other',
     'CV19-I-O-73': 'Other',
     'CV19-I-O-34': 'Other',
     'CV19-I-O-59': 'Other',
     'CV19-I-O-71': 'Other',
     'CV19-I-O-74': 'Other',
     'CV19-I-O-4': 'RA',
     'CV19-I-O-39': 'RA',
     'CV19-I-O-40': 'RA',
     'CV19-I-O-45': 'RA',
     'CV19-I-O-51': 'RA',
     'CV19-I-O-14': 'Other',
     'CV19-I-O-16': 'Other',
     'CV19-I-O-17': 'Other',
     'CV19-I-O-18': 'Other',
     'CV19-I-O-19': 'Other',
     'CV19-I-O-20': 'Other',
     'CV19-I-O-21': 'Other',
     'CV19-I-O-22': 'Other',
     'CV19-I-O-23': 'Other',
     'CV19-I-O-24': 'Other'}
adata.obs['Immune_condition_RA'] = adata.obs['demultiplexed'].map(donor2condition).astype('category')
adata.obs['celltype_condition_RA'] = [i + '_' + j for i,j in zip(adata.obs['Final_Celltype'], adata.obs['Immune_condition_RA'])]

donor2condition = {
     'CV19-I-O-2': 'MS',
     'CV19-I-O-3': 'MS',
     'CV19-I-O-73': 'MS',
     'CV19-I-O-34': 'Other',
     'CV19-I-O-59': 'Other',
     'CV19-I-O-71': 'Other',
     'CV19-I-O-74': 'Other',
     'CV19-I-O-4': 'Other',
     'CV19-I-O-39': 'Other',
     'CV19-I-O-40': 'Other',
     'CV19-I-O-45': 'Other',
     'CV19-I-O-51': 'Other',
     'CV19-I-O-14': 'Other',
     'CV19-I-O-16': 'Other',
     'CV19-I-O-17': 'Other',
     'CV19-I-O-18': 'Other',
     'CV19-I-O-19': 'Other',
     'CV19-I-O-20': 'Other',
     'CV19-I-O-21': 'Other',
     'CV19-I-O-22': 'Other',
     'CV19-I-O-23': 'Other',
     'CV19-I-O-24': 'Other'}
adata.obs['Immune_condition_MS'] = adata.obs['demultiplexed'].map(donor2condition).astype('category')
adata.obs['celltype_condition_MS'] = [i + '_' + j for i,j in zip(adata.obs['Final_Celltype'], adata.obs['Immune_condition_MS'])]

donor2condition = {
     'CV19-I-O-2': 'Other',
     'CV19-I-O-3': 'Other',
     'CV19-I-O-73': 'Other',
     'CV19-I-O-34': 'PS',
     'CV19-I-O-59': 'PS',
     'CV19-I-O-71': 'PS',
     'CV19-I-O-74': 'PS',
     'CV19-I-O-4': 'Other',
     'CV19-I-O-39': 'Other',
     'CV19-I-O-40': 'Other',
     'CV19-I-O-45': 'Other',
     'CV19-I-O-51': 'Other',
     'CV19-I-O-14': 'Other',
     'CV19-I-O-16': 'Other',
     'CV19-I-O-17': 'Other',
     'CV19-I-O-18': 'Other',
     'CV19-I-O-19': 'Other',
     'CV19-I-O-20': 'Other',
     'CV19-I-O-21': 'Other',
     'CV19-I-O-22': 'Other',
     'CV19-I-O-23': 'Other',
     'CV19-I-O-24': 'Other'}
adata.obs['Immune_condition_PS'] = adata.obs['demultiplexed'].map(donor2condition).astype('category')
adata.obs['celltype_condition_PS'] = [i + '_' + j for i,j in zip(adata.obs['Final_Celltype'], adata.obs['Immune_condition_PS'])]

donor2condition = {
     'CV19-I-O-2': 'Diseased',
     'CV19-I-O-3': 'Diseased',
     'CV19-I-O-73': 'Diseased',
     'CV19-I-O-34': 'Diseased',
     'CV19-I-O-59': 'Diseased',
     'CV19-I-O-71': 'Diseased',
     'CV19-I-O-74': 'Diseased',
     'CV19-I-O-4': 'Diseased',
     'CV19-I-O-39': 'Diseased',
     'CV19-I-O-40': 'Diseased',
     'CV19-I-O-45': 'Diseased',
     'CV19-I-O-51': 'Diseased',
     'CV19-I-O-14': 'Control',
     'CV19-I-O-16': 'Control',
     'CV19-I-O-17': 'Control',
     'CV19-I-O-18': 'Control',
     'CV19-I-O-19': 'Control',
     'CV19-I-O-20': 'Control',
     'CV19-I-O-21': 'Control',
     'CV19-I-O-22': 'Control',
     'CV19-I-O-23': 'Control',
     'CV19-I-O-24': 'Control'}
adata.obs['Immune_condition_Control'] = adata.obs['demultiplexed'].map(donor2condition).astype('category')
adata.obs['celltype_condition_Control'] = [i + '_' + j for i,j in zip(adata.obs['Final_Celltype'], adata.obs['Immune_condition_Control'])]

adata.obs

Trying to set attribute `.obs` of view, copying.


,souporcell,demultiplexed,sample_names,log2p1_count,percent_mito,n_genes,batch,FolderName,Lane,Sort,...,Immune_condition,celltype_condition,Immune_condition_RA,celltype_condition_RA,Immune_condition_MS,celltype_condition_MS,Immune_condition_PS,celltype_condition_PS,Immune_condition_Control,celltype_condition_Control
RV8919578_AAACCTGAGAAACCTA-1,singlet,CV19-I-O-3,RV8919578,11.406736,0.017318,1159,2,I-O-2_3_C-0,34527_1,None,...,MS,NK CD56(bright)_MS,Other,NK CD56(bright)_Other,MS,NK CD56(bright)_MS,Other,NK CD56(bright)_Other,Diseased,NK CD56(bright)_Diseased
RV8919578_AAACCTGAGAGCAATT-1,singlet,CV19-I-O-3,RV8919578,12.939213,0.026614,2058,2,I-O-2_3_C-0,34527_1,None,...,MS,Naïve B_MS,Other,Naïve B_Other,MS,Naïve B_MS,Other,Naïve B_Other,Diseased,Naïve B_Diseased
RV8919578_AAACCTGAGAGCTGGT-1,singlet,CV19-I-O-2,RV8919578,13.342491,0.007992,2597,2,I-O-2_3_C-0,34527_1,None,...,MS,pDC_MS,Other,pDC_Other,MS,pDC_MS,Other,pDC_Other,Diseased,pDC_Diseased
RV8919578_AAACCTGAGCGAAGGG-1,singlet,CV19-I-O-3,RV8919578,12.463013,0.031887,1902,2,I-O-2_3_C-0,34527_1,None,...,MS,NK CD56(dim)_MS,Other,NK CD56(dim)_Other,MS,NK CD56(dim)_MS,Other,NK CD56(dim)_Other,Diseased,NK CD56(dim)_Diseased
RV8919578_AAACCTGAGCGATGAC-1,singlet,CV19-I-O-2,RV8919578,11.874598,0.034896,882,2,I-O-2_3_C-0,34527_1,None,...,MS,NK CD56(bright)_MS,Other,NK CD56(bright)_Other,MS,NK CD56(bright)_MS,Other,NK CD56(bright)_Other,Diseased,NK CD56(bright)_Diseased
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RV8959686_TTTGTCAGTATGAATG-1,singlet,CV19-I-O-71,RV8959686,11.049168,0.131728,689,24,I-O-71_C-0,38186,None,...,PS,CD4 naïve T_PS,Other,CD4 naïve T_Other,Other,CD4 naïve T_Other,PS,CD4 naïve T_PS,Diseased,CD4 naïve T_Diseased
RV8959686_TTTGTCAGTCGGCATC-1,singlet,CV19-I-O-71,RV8959686,11.604091,0.026028,1058,24,I-O-71_C-0,38186,None,...,PS,Naïve B_PS,Other,Naïve B_Other,Other,Naïve B_Other,PS,Naïve B_PS,Diseased,Naïve B_Diseased
RV8959686_TTTGTCAGTGACAAAT-1,singlet,CV19-I-O-71,RV8959686,11.200898,0.064173,792,24,I-O-71_C-0,38186,None,...,PS,NK CD56(bright)_PS,Other,NK CD56(bright)_Other,Other,NK CD56(bright)_Other,PS,NK CD56(bright)_PS,Diseased,NK CD56(bright)_Diseased
RV8959686_TTTGTCAGTTGCCTCT-1,singlet,CV19-I-O-71,RV8959686,12.747355,0.008582,1535,24,I-O-71_C-0,38186,None,...,PS,CD4 naïve T_PS,Other,CD4 naïve T_Other,Other,CD4 naïve T_Other,PS,CD4 naïve T_PS,Diseased,CD4 naïve T_Diseased


In [9]:
np.unique(adata.obs["Final_Celltype"], return_counts = True)

(array(['CD14 mono', 'CD16 mono', 'CD4 memory T', 'CD4 naïve T',
        'CD8 memory T', 'CD8 naïve T', 'Exhausted B', 'HSC', 'Immature B',
        'MAIT', 'Memory B', 'NK CD56(bright)', 'NK CD56(dim)', 'NKT',
        'Naïve B', 'Neutrophil', 'Plasma B', 'Plasmablast', 'Platelets',
        'Prolif NK', 'Prolif T', 'RBC', 'Treg', 'cDC1', 'cDC2', 'cDC3',
        'pDC', 'γδT'], dtype=object),
 array([23648,  1923,  3276, 26887,  6224,  2387,   510,   270,   255,
          223,   457,  3638,  6948,  1807,  8679,   299,   415,  1239,
         1444,   226,  1697,  3907,   306,    23,   254,   266,    75,
          216]))

In [10]:
# change for each cell type (example here NK CD56(dim))
adata_nkbright = adata[adata.obs['Final_Celltype'].isin(["NK CD56(dim)"])].copy()
adata_nkbright

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


AnnData object with n_obs × n_vars = 6948 × 22553
    obs: 'souporcell', 'demultiplexed', 'sample_names', 'log2p1_count', 'percent_mito', 'n_genes', 'batch', 'FolderName', 'Lane', 'Sort', 'Type', 'Donor Id', 'Age', 'Sex', 'Race', 'Ethnicity', 'BMI', 'Pre-existing heart disease', 'Pre-existing lung disease', 'Pre-existing kidney disease', 'Pre-existing diabetes', 'Pre-existing Hypertension', 'Pre-existing immunocompromised condition', 'Smoking', 'SARS-CoV-2 PCR', 'SARS-CoV-2 Ab', 'Symptomatic', 'Admitted to hospital', 'Highest level of respiratory support', 'Vasoactive agents required during hospitalization', '28-day death', 'scrublet_pred', 'scrublet_local_pred', 'scrublet_score', 'scrublet_cluster_score', 'filtered_cells', 'S_score', 'G2M_score', 'phase', 'leiden_sampl_cc', 'leidenres2_sampl_cc', 'DonorSubset', 'leiden_scvi_subset_cc', 'leidenres2_scvi_subset_cc', 'leiden_tvi_subset_cc', 'leidenres2_tvi_subset_cc', 'louvain', 'Celltype_Predictions', 'Prediction_Probabilities', 'Final_

In [11]:
# process reference dataset similarly and filter
adata_ref = anndata.AnnData(X = adata_ref.raw.X, obs = adata_ref.obs, var = adata_ref.raw.var, obsm = adata_ref.obsm).copy()
sc.pp.filter_genes(adata_ref, min_cells = 3)
sc.pp.normalize_per_cell(adata_ref, counts_per_cell_after = 1e4)
sc.pp.log1p(adata_ref)
adata_ref

filtered out 6748 genes that are detected in less than 3 cells
normalizing by total count per cell
    finished (0:00:12): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)


AnnData object with n_obs × n_vars = 691683 × 24339
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'Resample', 'Collection_Day', 'Sex', 'Age', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'initial_clustering', 'full_clustering', 'n_counts'
    var: 'feature_types', 'n_cells'
    uns: 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [13]:
all_genes_ref = adata_ref.var.index
genes_to_remove_ref = list(set(all_genes_ref) & set(Y_chr))
all_genes_ref = [ele for ele in all_genes_ref if ele not in genes_to_remove_ref]
adata_ref = adata_ref[: , all_genes_ref]
adata_ref

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


View of AnnData object with n_obs × n_vars = 691683 × 24315
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'Resample', 'Collection_Day', 'Sex', 'Age', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'initial_clustering', 'full_clustering', 'n_counts'
    var: 'feature_types', 'n_cells'
    uns: 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [14]:
adata_ref_COVID = adata_ref[adata_ref.obs["Status_on_day_collection_summary"].isin(['Moderate', 'Critical', 'Severe'])].copy()
adata_ref_COVID

AnnData object with n_obs × n_vars = 365245 × 24315
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'Resample', 'Collection_Day', 'Sex', 'Age', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'initial_clustering', 'full_clustering', 'n_counts'
    var: 'feature_types', 'n_cells'
    uns: 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [15]:
np.unique(adata_ref_COVID.obs["full_clustering"])

array(['ASDC', 'B_exhausted', 'B_immature', 'B_malignant', 'B_naive',
       'B_non-switched_memory', 'B_switched_memory', 'C1_CD16_mono',
       'CD14_mono', 'CD16_mono', 'CD4.CM', 'CD4.EM', 'CD4.IL22',
       'CD4.Naive', 'CD4.Prolif', 'CD4.Tfh', 'CD4.Th1', 'CD4.Th17',
       'CD4.Th2', 'CD8.EM', 'CD8.Naive', 'CD8.Prolif', 'CD8.TE',
       'CD83_CD14_mono', 'DC1', 'DC2', 'DC3', 'DC_prolif', 'HSC',
       'ILC1_3', 'ILC2', 'MAIT', 'Mono_prolif', 'NKT', 'NK_16hi',
       'NK_56hi', 'NK_prolif', 'Plasma_cell_IgA', 'Plasma_cell_IgG',
       'Plasma_cell_IgM', 'Plasmablast', 'Platelets', 'RBC', 'Treg',
       'gdT', 'pDC'], dtype=object)

In [16]:
# change for each cell type (example here NK CD56(dim)/NK_16hi)
adata_ref_nkbright = adata_ref_COVID[adata_ref_COVID.obs['full_clustering'].isin(['NK_16hi'])]
adata_ref_nkbright

View of AnnData object with n_obs × n_vars = 49571 × 24315
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'Resample', 'Collection_Day', 'Sex', 'Age', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'initial_clustering', 'full_clustering', 'n_counts'
    var: 'feature_types', 'n_cells'
    uns: 'log1p'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'

In [17]:
adata_ref_nkbright = adata_ref_nkbright[adata_ref_nkbright.obs['Site'].isin(['Sanger'])].copy()
np.unique(adata_ref_nkbright.obs["Sex"], return_counts = True)

(array(['Female', 'Male'], dtype=object), array([5329, 2744]))

limma DE Analysis

In [18]:
%load_ext rpy2.ipython

RA

In [19]:
# create dataframes to be used for analysis
ctrl = 'NK CD56(dim)_Other'
case = 'NK CD56(dim)_RA'

t = adata_nkbright.X.toarray().T
df = pd.DataFrame(data = t, columns = adata_nkbright.obs.index, index = adata_nkbright.var_names)

meta_df = pd.DataFrame(data={'Cell':list(adata_nkbright.obs.index),
                             'Celltype':[str(i) for i in adata_nkbright.obs['celltype_condition_RA']],
                             'Sample':[str(i) for i in adata_nkbright.obs['demultiplexed']]})
meta_df.set_index('Cell', inplace = True)
meta_df.reset_index(inplace = True)
meta_df

,Cell,Celltype,Sample
0,RV8919578_AAACCTGAGCGAAGGG-1,NK CD56(dim)_Other,CV19-I-O-3
1,RV8919578_AAACCTGAGCTGTTCA-1,NK CD56(dim)_Other,CV19-I-O-3
2,RV8919578_AAACCTGCAGCCAATT-1,NK CD56(dim)_Other,CV19-I-O-2
3,RV8919578_AAACCTGGTTCCACGG-1,NK CD56(dim)_Other,CV19-I-O-2
4,RV8919578_AAACGGGGTCATATGC-1,NK CD56(dim)_Other,CV19-I-O-3
...,...,...,...
6943,RV8959686_TTTATGCAGGAGTAGA-1,NK CD56(dim)_Other,CV19-I-O-71
6944,RV8959686_TTTATGCCAGCGATCC-1,NK CD56(dim)_Other,CV19-I-O-71
6945,RV8959686_TTTCCTCGTGCGCTTG-1,NK CD56(dim)_Other,CV19-I-O-71
6946,RV8959686_TTTCCTCTCATGTGGT-1,NK CD56(dim)_Other,CV19-I-O-71


In [20]:
np.unique(meta_df.Celltype)

array(['NK CD56(dim)_Other', 'NK CD56(dim)_RA'], dtype=object)

In [21]:
%%R -i df -i meta_df -i ctrl -i case

#Use edgeR to import, filter, and normalise data then limma (voom) for linear modelling and empirical Bayes moderation to assess DE
library(limma)
library(edgeR)

#Format
ex_mat = as.matrix(df)
rownames(meta_df) = meta_df$Cell

print(unique(meta_df$Celltype))

#Shared cells
shared_cells = intersect(rownames(meta_df), colnames(ex_mat))
message(length(shared_cells), 'shared cells')
ex_mat = ex_mat[, shared_cells]
meta_df = meta_df[shared_cells,]

#Filter lowly expressed genes
keep = rowSums(ex_mat, na.rm = T) > 0.1
ex_mat = ex_mat[keep, ]
keep = aveLogCPM(ex_mat) > 0.1
ex_mat = ex_mat[keep, ]

#Extract celltypes
cells = rownames(meta_df)
celltypes = unique(meta_df$Celltype)
covariates = meta_df$covariate

#Extract cells according to condition
cells_ctrl = rownames(subset(meta_df, Celltype == ctrl))
cells_case = rownames(subset(meta_df, Celltype == case))

#Build cluster_type vector
cluster_type = rep(0, length(cells))
names(cluster_type) = cells
cluster_type[cells_ctrl] = 'ctrl'
cluster_type[cells_case] = 'case'

print(unique(cluster_type))
design.matrix <- model.matrix(~ 0 + cluster_type)

#Define the comparison (i.e., case vs control)
contrast.matrix <- makeContrasts(caseVScontrol = cluster_typecase - cluster_typectrl, levels = design.matrix)

#Make model and run contrasts
fit <- lmFit(ex_mat, design.matrix)
fit <- contrasts.fit(fit, contrast.matrix)
fit <- eBayes(fit)

#Make a dataframe containing the data
results = topTable(fit, adjust="fdr", number = nrow(ex_mat), coef = 'caseVScontrol')

#Add and filter desired data
results$Gene = rownames(results)
results = results[ , c('Gene', 'logFC', 'P.Value', 'adj.P.Val')]
results$AveExpr_case = apply(ex_mat[results$Gene, cells_case], 1, mean)
results$AveExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, mean)
results$percentExpr_case = apply(ex_mat[results$Gene, cells_case], 1, function(x) sum(c(x > 0)+0) ) / length(cells_case)
results$percentExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, function(x) sum(c(x > 0)+0) ) / length(cells_ctrl)
results$AveExpr_case = round(results$AveExpr_case, 6)
results$AveExpr_ctrl = round(results$AveExpr_ctrl, 6)
results$percentExpr_case = round(results$percentExpr_case, 6)
results$percentExpr_ctrl = round(results$percentExpr_ctrl, 6)

#Store results as a .csv file
write.csv(results, file = paste0('./', ctrl, '_vs_', case, '.csv'), row.names = F, col.names = T, quote = F)

[1] "NK CD56(dim)_Other" "NK CD56(dim)_RA"   


R[write to console]: 6948shared cells



[1] "ctrl" "case"


PS

In [22]:
# create dataframes to be used for analysis
ctrl = 'NK CD56(dim)_Other'
case = 'NK CD56(dim)_PS'

t = adata_nkbright.X.toarray().T
df = pd.DataFrame(data = t, columns = adata_nkbright.obs.index, index = adata_nkbright.var_names)

meta_df = pd.DataFrame(data={'Cell':list(adata_nkbright.obs.index),
                             'Celltype':[str(i) for i in adata_nkbright.obs['celltype_condition_PS']],
                             'Sample':[str(i) for i in adata_nkbright.obs['demultiplexed']]})
meta_df.set_index('Cell', inplace = True)
meta_df.reset_index(inplace = True)
meta_df

,Cell,Celltype,Sample
0,RV8919578_AAACCTGAGCGAAGGG-1,NK CD56(dim)_Other,CV19-I-O-3
1,RV8919578_AAACCTGAGCTGTTCA-1,NK CD56(dim)_Other,CV19-I-O-3
2,RV8919578_AAACCTGCAGCCAATT-1,NK CD56(dim)_Other,CV19-I-O-2
3,RV8919578_AAACCTGGTTCCACGG-1,NK CD56(dim)_Other,CV19-I-O-2
4,RV8919578_AAACGGGGTCATATGC-1,NK CD56(dim)_Other,CV19-I-O-3
...,...,...,...
6943,RV8959686_TTTATGCAGGAGTAGA-1,NK CD56(dim)_PS,CV19-I-O-71
6944,RV8959686_TTTATGCCAGCGATCC-1,NK CD56(dim)_PS,CV19-I-O-71
6945,RV8959686_TTTCCTCGTGCGCTTG-1,NK CD56(dim)_PS,CV19-I-O-71
6946,RV8959686_TTTCCTCTCATGTGGT-1,NK CD56(dim)_PS,CV19-I-O-71


In [23]:
%%R -i df -i meta_df -i ctrl -i case

#Use edgeR to import, filter, and normalise data then limma (voom) for linear modelling and empirical Bayes moderation to assess DE
library(limma)
library(edgeR)

#Format
ex_mat = as.matrix(df)
rownames(meta_df) = meta_df$Cell

print(unique(meta_df$Celltype))

#Shared cells
shared_cells = intersect(rownames(meta_df), colnames(ex_mat))
message(length(shared_cells), 'shared cells')
ex_mat = ex_mat[, shared_cells]
meta_df = meta_df[shared_cells,]

#Filter lowly expressed genes
keep = rowSums(ex_mat, na.rm = T) > 0.1
ex_mat = ex_mat[keep, ]
keep = aveLogCPM(ex_mat) > 0.1
ex_mat = ex_mat[keep, ]

#Extract celltypes
cells = rownames(meta_df)
celltypes = unique(meta_df$Celltype)
covariates = meta_df$covariate

#Extract cells according to condition
cells_ctrl = rownames(subset(meta_df, Celltype == ctrl))
cells_case = rownames(subset(meta_df, Celltype == case))

#Build cluster_type vector
cluster_type = rep(0, length(cells))
names(cluster_type) = cells
cluster_type[cells_ctrl] = 'ctrl'
cluster_type[cells_case] = 'case'

print(unique(cluster_type))
design.matrix <- model.matrix(~ 0 + cluster_type)

#Define the comparison (i.e., case vs control)
contrast.matrix <- makeContrasts(caseVScontrol = cluster_typecase - cluster_typectrl, levels = design.matrix)

#Make model and run contrasts
fit <- lmFit(ex_mat, design.matrix)
fit <- contrasts.fit(fit, contrast.matrix)
fit <- eBayes(fit)

#Make a dataframe containing the data
results = topTable(fit, adjust="fdr", number = nrow(ex_mat), coef = 'caseVScontrol')

#Add and filter desired data
results$Gene = rownames(results)
results = results[ , c('Gene', 'logFC', 'P.Value', 'adj.P.Val')]
results$AveExpr_case = apply(ex_mat[results$Gene, cells_case], 1, mean)
results$AveExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, mean)
results$percentExpr_case = apply(ex_mat[results$Gene, cells_case], 1, function(x) sum(c(x > 0)+0) ) / length(cells_case)
results$percentExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, function(x) sum(c(x > 0)+0) ) / length(cells_ctrl)
results$AveExpr_case = round(results$AveExpr_case, 6)
results$AveExpr_ctrl = round(results$AveExpr_ctrl, 6)
results$percentExpr_case = round(results$percentExpr_case, 6)
results$percentExpr_ctrl = round(results$percentExpr_ctrl, 6)

#Store results as a .csv file
write.csv(results, file = paste0('./', ctrl, '_vs_', case, '.csv'), row.names = F, col.names = T, quote = F)

[1] "NK CD56(dim)_Other" "NK CD56(dim)_PS"   


R[write to console]: 6948shared cells



[1] "ctrl" "case"


MS

In [24]:
# create dataframes to be used for analysis
ctrl = 'NK CD56(dim)_Other'
case = 'NK CD56(dim)_MS'

t = adata_nkbright.X.toarray().T
df = pd.DataFrame(data = t, columns = adata_nkbright.obs.index, index = adata_nkbright.var_names)

meta_df = pd.DataFrame(data={'Cell':list(adata_nkbright.obs.index),
                             'Celltype':[str(i) for i in adata_nkbright.obs['celltype_condition_MS']],
                             'Sample':[str(i) for i in adata_nkbright.obs['demultiplexed']]})
meta_df.set_index('Cell', inplace = True)
meta_df.reset_index(inplace = True)
meta_df

,Cell,Celltype,Sample
0,RV8919578_AAACCTGAGCGAAGGG-1,NK CD56(dim)_MS,CV19-I-O-3
1,RV8919578_AAACCTGAGCTGTTCA-1,NK CD56(dim)_MS,CV19-I-O-3
2,RV8919578_AAACCTGCAGCCAATT-1,NK CD56(dim)_MS,CV19-I-O-2
3,RV8919578_AAACCTGGTTCCACGG-1,NK CD56(dim)_MS,CV19-I-O-2
4,RV8919578_AAACGGGGTCATATGC-1,NK CD56(dim)_MS,CV19-I-O-3
...,...,...,...
6943,RV8959686_TTTATGCAGGAGTAGA-1,NK CD56(dim)_Other,CV19-I-O-71
6944,RV8959686_TTTATGCCAGCGATCC-1,NK CD56(dim)_Other,CV19-I-O-71
6945,RV8959686_TTTCCTCGTGCGCTTG-1,NK CD56(dim)_Other,CV19-I-O-71
6946,RV8959686_TTTCCTCTCATGTGGT-1,NK CD56(dim)_Other,CV19-I-O-71


In [25]:
%%R -i df -i meta_df -i ctrl -i case

#Use edgeR to import, filter, and normalise data then limma (voom) for linear modelling and empirical Bayes moderation to assess DE
library(limma)
library(edgeR)

#Format
ex_mat = as.matrix(df)
rownames(meta_df) = meta_df$Cell

print(unique(meta_df$Celltype))

#Shared cells
shared_cells = intersect(rownames(meta_df), colnames(ex_mat))
message(length(shared_cells), 'shared cells')
ex_mat = ex_mat[, shared_cells]
meta_df = meta_df[shared_cells,]

#Filter lowly expressed genes
keep = rowSums(ex_mat, na.rm = T) > 0.1
ex_mat = ex_mat[keep, ]
keep = aveLogCPM(ex_mat) > 0.1
ex_mat = ex_mat[keep, ]

#Extract celltypes
cells = rownames(meta_df)
celltypes = unique(meta_df$Celltype)
covariates = meta_df$covariate

#Extract cells according to condition
cells_ctrl = rownames(subset(meta_df, Celltype == ctrl))
cells_case = rownames(subset(meta_df, Celltype == case))

#Build cluster_type vector
cluster_type = rep(0, length(cells))
names(cluster_type) = cells
cluster_type[cells_ctrl] = 'ctrl'
cluster_type[cells_case] = 'case'

print(unique(cluster_type))
design.matrix <- model.matrix(~ 0 + cluster_type)

#Define the comparison (i.e., case vs control)
contrast.matrix <- makeContrasts(caseVScontrol = cluster_typecase - cluster_typectrl, levels = design.matrix)

#Make model and run contrasts
fit <- lmFit(ex_mat, design.matrix)
fit <- contrasts.fit(fit, contrast.matrix)
fit <- eBayes(fit)

#Make a dataframe containing the data
results = topTable(fit, adjust="fdr", number = nrow(ex_mat), coef = 'caseVScontrol')

#Add and filter desired data
results$Gene = rownames(results)
results = results[ , c('Gene', 'logFC', 'P.Value', 'adj.P.Val')]
results$AveExpr_case = apply(ex_mat[results$Gene, cells_case], 1, mean)
results$AveExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, mean)
results$percentExpr_case = apply(ex_mat[results$Gene, cells_case], 1, function(x) sum(c(x > 0)+0) ) / length(cells_case)
results$percentExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, function(x) sum(c(x > 0)+0) ) / length(cells_ctrl)
results$AveExpr_case = round(results$AveExpr_case, 6)
results$AveExpr_ctrl = round(results$AveExpr_ctrl, 6)
results$percentExpr_case = round(results$percentExpr_case, 6)
results$percentExpr_ctrl = round(results$percentExpr_ctrl, 6)

#Store results as a .csv file
write.csv(results, file = paste0('./', ctrl, '_vs_', case, '.csv'), row.names = F, col.names = T, quote = F)

[1] "NK CD56(dim)_MS"    "NK CD56(dim)_Other"


R[write to console]: 6948shared cells



[1] "case" "ctrl"


Control

In [26]:
# create dataframes to be used for analysis
ctrl = 'NK CD56(dim)_Control'
case = 'NK CD56(dim)_Diseased'

t = adata_nkbright.X.toarray().T
df = pd.DataFrame(data = t, columns = adata_nkbright.obs.index, index = adata_nkbright.var_names)

meta_df = pd.DataFrame(data={'Cell':list(adata_nkbright.obs.index),
                             'Celltype':[str(i) for i in adata_nkbright.obs['celltype_condition_Control']],
                             'Sample':[str(i) for i in adata_nkbright.obs['demultiplexed']]})
meta_df.set_index('Cell', inplace = True)
meta_df.reset_index(inplace = True)
meta_df

,Cell,Celltype,Sample
0,RV8919578_AAACCTGAGCGAAGGG-1,NK CD56(dim)_Diseased,CV19-I-O-3
1,RV8919578_AAACCTGAGCTGTTCA-1,NK CD56(dim)_Diseased,CV19-I-O-3
2,RV8919578_AAACCTGCAGCCAATT-1,NK CD56(dim)_Diseased,CV19-I-O-2
3,RV8919578_AAACCTGGTTCCACGG-1,NK CD56(dim)_Diseased,CV19-I-O-2
4,RV8919578_AAACGGGGTCATATGC-1,NK CD56(dim)_Diseased,CV19-I-O-3
...,...,...,...
6943,RV8959686_TTTATGCAGGAGTAGA-1,NK CD56(dim)_Diseased,CV19-I-O-71
6944,RV8959686_TTTATGCCAGCGATCC-1,NK CD56(dim)_Diseased,CV19-I-O-71
6945,RV8959686_TTTCCTCGTGCGCTTG-1,NK CD56(dim)_Diseased,CV19-I-O-71
6946,RV8959686_TTTCCTCTCATGTGGT-1,NK CD56(dim)_Diseased,CV19-I-O-71


In [27]:
%%R -i df -i meta_df -i ctrl -i case

#Use edgeR to import, filter, and normalise data then limma (voom) for linear modelling and empirical Bayes moderation to assess DE
library(limma)
library(edgeR)

#Format
ex_mat = as.matrix(df)
rownames(meta_df) = meta_df$Cell

print(unique(meta_df$Celltype))

#Shared cells
shared_cells = intersect(rownames(meta_df), colnames(ex_mat))
message(length(shared_cells), 'shared cells')
ex_mat = ex_mat[, shared_cells]
meta_df = meta_df[shared_cells,]

#Filter lowly expressed genes
keep = rowSums(ex_mat, na.rm = T) > 0.1
ex_mat = ex_mat[keep, ]
keep = aveLogCPM(ex_mat) > 0.1
ex_mat = ex_mat[keep, ]

#Extract celltypes
cells = rownames(meta_df)
celltypes = unique(meta_df$Celltype)
covariates = meta_df$covariate

#Extract cells according to condition
cells_ctrl = rownames(subset(meta_df, Celltype == ctrl))
cells_case = rownames(subset(meta_df, Celltype == case))

#Build cluster_type vector
cluster_type = rep(0, length(cells))
names(cluster_type) = cells
cluster_type[cells_ctrl] = 'ctrl'
cluster_type[cells_case] = 'case'

print(unique(cluster_type))
design.matrix <- model.matrix(~ 0 + cluster_type)

#Define the comparison (i.e., case vs control)
contrast.matrix <- makeContrasts(caseVScontrol = cluster_typecase - cluster_typectrl, levels = design.matrix)

#Make model and run contrasts
fit <- lmFit(ex_mat, design.matrix)
fit <- contrasts.fit(fit, contrast.matrix)
fit <- eBayes(fit)

#Make a dataframe containing the data
results = topTable(fit, adjust="fdr", number = nrow(ex_mat), coef = 'caseVScontrol')

#Add and filter desired data
results$Gene = rownames(results)
results = results[ , c('Gene', 'logFC', 'P.Value', 'adj.P.Val')]
results$AveExpr_case = apply(ex_mat[results$Gene, cells_case], 1, mean)
results$AveExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, mean)
results$percentExpr_case = apply(ex_mat[results$Gene, cells_case], 1, function(x) sum(c(x > 0)+0) ) / length(cells_case)
results$percentExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, function(x) sum(c(x > 0)+0) ) / length(cells_ctrl)
results$AveExpr_case = round(results$AveExpr_case, 6)
results$AveExpr_ctrl = round(results$AveExpr_ctrl, 6)
results$percentExpr_case = round(results$percentExpr_case, 6)
results$percentExpr_ctrl = round(results$percentExpr_ctrl, 6)

#Store results as a .csv file
write.csv(results, file = paste0('./', ctrl, '_vs_', case, '.csv'), row.names = F, col.names = T, quote = F)

[1] "NK CD56(dim)_Diseased" "NK CD56(dim)_Control" 


R[write to console]: 6948shared cells



[1] "case" "ctrl"


Ref M/F

In [28]:
# create dataframes to be used for analysis
ctrl = 'Male'
case = 'Female'

t = adata_ref_nkbright.X.toarray().T
df = pd.DataFrame(data = t, columns = adata_ref_nkbright.obs.index, index = adata_ref_nkbright.var_names)

meta_df = pd.DataFrame(data={'Cell':list(adata_ref_nkbright.obs.index),
                             'Celltype':[str(i) for i in adata_ref_nkbright.obs['Sex']]})
meta_df.set_index('Cell', inplace = True)
meta_df.reset_index(inplace = True)
meta_df

,Cell,Celltype
0,S9_AAAGCAAAGACTAGAT-1,Male
1,S9_AAAGCAAAGGTGCTAG-1,Female
2,S9_AAAGCAATCATTATCC-1,Male
3,S9_AACCATGGTTGAGTTC-1,Male
4,S9_AACTCTTGTGCTGTAT-1,Male
...,...,...
8068,S24_TTTACTGGTATGAATG-1,Female
8069,S24_TTTCCTCCACACGCTG-1,Female
8070,S24_TTTGCGCTCGCAAACT-1,Female
8071,S24_TTTGGTTAGTGTCCAT-1,Female


In [29]:
%%R -i df -i meta_df -i ctrl -i case

#Use edgeR to import, filter, and normalise data then limma (voom) for linear modelling and empirical Bayes moderation to assess DE
library(limma)
library(edgeR)

#Format
ex_mat = as.matrix(df)
rownames(meta_df) = meta_df$Cell

print(unique(meta_df$Celltype))

#Shared cells
shared_cells = intersect(rownames(meta_df), colnames(ex_mat))
message(length(shared_cells), 'shared cells')
ex_mat = ex_mat[, shared_cells]
meta_df = meta_df[shared_cells,]

#Filter lowly expressed genes
keep = rowSums(ex_mat, na.rm = T) > 0.1
ex_mat = ex_mat[keep, ]
keep = aveLogCPM(ex_mat) > 0.1
ex_mat = ex_mat[keep, ]

#Extract celltypes
cells = rownames(meta_df)
celltypes = unique(meta_df$Celltype)
covariates = meta_df$covariate

#Extract cells according to condition
cells_ctrl = rownames(subset(meta_df, Celltype == ctrl))
cells_case = rownames(subset(meta_df, Celltype == case))

#Build cluster_type vector
cluster_type = rep(0, length(cells))
names(cluster_type) = cells
cluster_type[cells_ctrl] = 'ctrl'
cluster_type[cells_case] = 'case'

print(unique(cluster_type))
design.matrix <- model.matrix(~ 0 + cluster_type)

#Define the comparison (i.e., case vs control)
contrast.matrix <- makeContrasts(caseVScontrol = cluster_typecase - cluster_typectrl, levels = design.matrix)

#Make model and run contrasts
fit <- lmFit(ex_mat, design.matrix)
fit <- contrasts.fit(fit, contrast.matrix)
fit <- eBayes(fit)

#Make a dataframe containing the data
results = topTable(fit, adjust="fdr", number = nrow(ex_mat), coef = 'caseVScontrol')

#Add and filter desired data
results$Gene = rownames(results)
results = results[ , c('Gene', 'logFC', 'P.Value', 'adj.P.Val')]
results$AveExpr_case = apply(ex_mat[results$Gene, cells_case], 1, mean)
results$AveExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, mean)
results$percentExpr_case = apply(ex_mat[results$Gene, cells_case], 1, function(x) sum(c(x > 0)+0) ) / length(cells_case)
results$percentExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, function(x) sum(c(x > 0)+0) ) / length(cells_ctrl)
results$AveExpr_case = round(results$AveExpr_case, 6)
results$AveExpr_ctrl = round(results$AveExpr_ctrl, 6)
results$percentExpr_case = round(results$percentExpr_case, 6)
results$percentExpr_ctrl = round(results$percentExpr_ctrl, 6)

#Store results as a .csv file
write.csv(results, file = paste0('./NK CD56(dim)_', ctrl, '_vs_', case, '.csv'), row.names = F, col.names = T, quote = F)

[1] "Male"   "Female"


R[write to console]: 8073shared cells



[1] "ctrl" "case"


Ref COVID vs healthy

In [30]:
# subset and format reference object for comparison
adata_ref_cam = adata_ref[adata_ref.obs['Site'].isin(['Cambridge'])].copy()
adata_ref_all = adata_ref_cam[adata_ref_cam.obs['Status_on_day_collection_summary'].isin(['Critical', 'Healthy', 'Moderate'])].copy()
donor2condition = {
     'Critical': 'COVID',
     'Healthy': 'Healthy',
     'Moderate': 'COVID'}
adata_ref_all.obs['Condition'] = adata_ref_all.obs['Status_on_day_collection_summary'].map(donor2condition).astype('category')
adata_ref_all.obs['celltype_condition'] = [i + '_' + j for i,j in zip(adata_ref_all.obs['full_clustering'], adata_ref_all.obs['Condition'])]

adata_ref_all_nkbright = adata_ref_all[adata_ref_all.obs['full_clustering'].isin(['NK_16hi'])].copy()
donor2condition = {
     'NK_16hi_COVID': 'NK CD56(dim)_COVID',
     'NK_16hi_Healthy': 'NK CD56(dim)_Healthy'}
adata_ref_all_nkbright.obs['celltype_condition_nkbright'] = adata_ref_all_nkbright.obs['celltype_condition'].map(donor2condition).astype('category')
np.unique(adata_ref_all_nkbright.obs['celltype_condition_nkbright'], return_counts = True)

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


(array(['NK CD56(dim)_COVID', 'NK CD56(dim)_Healthy'], dtype=object),
 array([3625, 3259]))

In [31]:
np.unique(adata_ref_all.obs['full_clustering'], return_counts = True)

(array(['ASDC', 'B_exhausted', 'B_immature', 'B_malignant', 'B_naive',
        'B_non-switched_memory', 'B_switched_memory', 'C1_CD16_mono',
        'CD14_mono', 'CD16_mono', 'CD4.CM', 'CD4.EM', 'CD4.IL22',
        'CD4.Naive', 'CD4.Prolif', 'CD4.Tfh', 'CD4.Th1', 'CD4.Th17',
        'CD4.Th2', 'CD8.EM', 'CD8.Naive', 'CD8.Prolif', 'CD8.TE',
        'CD83_CD14_mono', 'DC1', 'DC2', 'DC3', 'DC_prolif', 'HSC',
        'ILC1_3', 'ILC2', 'MAIT', 'Mono_prolif', 'NKT', 'NK_16hi',
        'NK_56hi', 'NK_prolif', 'Plasma_cell_IgA', 'Plasma_cell_IgG',
        'Plasma_cell_IgM', 'Plasmablast', 'Platelets', 'RBC', 'Treg',
        'gdT', 'pDC'], dtype=object),
 array([    7,   339,   634,  2313,  4949,   328,   832,     3,   256,
          340, 12305,   317,  4818,  6400,    39,  3073,   173,     2,
            9,  2178,  4654,    73,  3762,  3597,    35,   169,   427,
            2,   160,   208,    20,  2051,     7,   405,  6884,  1940,
          187,   297,   266,    37,   497,  2876,  1126,  1283

In [32]:
# create dataframes to be used for analysis
ctrl = 'NK CD56(dim)_Healthy'
case = 'NK CD56(dim)_COVID'

t = adata_ref_all_nkbright.X.toarray().T
df = pd.DataFrame(data = t, columns = adata_ref_all_nkbright.obs.index, index = adata_ref_all_nkbright.var_names)

meta_df = pd.DataFrame(data={'Cell':list(adata_ref_all_nkbright.obs.index),
                             'Celltype':[str(i) for i in adata_ref_all_nkbright.obs['celltype_condition_nkbright']]})
meta_df.set_index('Cell', inplace = True)
meta_df.reset_index(inplace = True)
meta_df

,Cell,Celltype
0,BGCV01_AAACCTGAGCTTATCG-1,NK CD56(dim)_Healthy
1,BGCV01_AAACGGGAGATCACGG-1,NK CD56(dim)_Healthy
2,BGCV01_AAAGCAACACGGTGTC-1,NK CD56(dim)_Healthy
3,BGCV01_AAAGCAAGTACCTACA-1,NK CD56(dim)_Healthy
4,BGCV01_AAAGCAAGTTGGTAAA-1,NK CD56(dim)_Healthy
...,...,...
6879,BGCV15_TTTACTGTCTCCAACC-1,NK CD56(dim)_COVID
6880,BGCV15_TTTCCTCCACTGTTAG-1,NK CD56(dim)_Healthy
6881,BGCV15_TTTGCGCTCACCGTAA-1,NK CD56(dim)_Healthy
6882,BGCV15_TTTGGTTTCAAGATCC-1,NK CD56(dim)_Healthy


In [33]:
%%R -i df -i meta_df -i ctrl -i case

#Use edgeR to import, filter, and normalise data then limma (voom) for linear modelling and empirical Bayes moderation to assess DE
library(limma)
library(edgeR)

#Format
ex_mat = as.matrix(df)
rownames(meta_df) = meta_df$Cell

print(unique(meta_df$Celltype))

#Shared cells
shared_cells = intersect(rownames(meta_df), colnames(ex_mat))
message(length(shared_cells), 'shared cells')
ex_mat = ex_mat[, shared_cells]
meta_df = meta_df[shared_cells,]

#Filter lowly expressed genes
keep = rowSums(ex_mat, na.rm = T) > 0.1
ex_mat = ex_mat[keep, ]
keep = aveLogCPM(ex_mat) > 0.1
ex_mat = ex_mat[keep, ]

#Extract celltypes
cells = rownames(meta_df)
celltypes = unique(meta_df$Celltype)
covariates = meta_df$covariate

#Extract cells according to condition
cells_ctrl = rownames(subset(meta_df, Celltype == ctrl))
cells_case = rownames(subset(meta_df, Celltype == case))

#Build cluster_type vector
cluster_type = rep(0, length(cells))
names(cluster_type) = cells
cluster_type[cells_ctrl] = 'ctrl'
cluster_type[cells_case] = 'case'

print(unique(cluster_type))
design.matrix <- model.matrix(~ 0 + cluster_type)

#Define the comparison (i.e., case vs control)
contrast.matrix <- makeContrasts(caseVScontrol = cluster_typecase - cluster_typectrl, levels = design.matrix)

#Make model and run contrasts
fit <- lmFit(ex_mat, design.matrix)
fit <- contrasts.fit(fit, contrast.matrix)
fit <- eBayes(fit)

#Make a dataframe containing the data
results = topTable(fit, adjust="fdr", number = nrow(ex_mat), coef = 'caseVScontrol')

#Add and filter desired data
results$Gene = rownames(results)
results = results[ , c('Gene', 'logFC', 'P.Value', 'adj.P.Val')]
results$AveExpr_case = apply(ex_mat[results$Gene, cells_case], 1, mean)
results$AveExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, mean)
results$percentExpr_case = apply(ex_mat[results$Gene, cells_case], 1, function(x) sum(c(x > 0)+0) ) / length(cells_case)
results$percentExpr_ctrl = apply(ex_mat[results$Gene, cells_ctrl], 1, function(x) sum(c(x > 0)+0) ) / length(cells_ctrl)
results$AveExpr_case = round(results$AveExpr_case, 6)
results$AveExpr_ctrl = round(results$AveExpr_ctrl, 6)
results$percentExpr_case = round(results$percentExpr_case, 6)
results$percentExpr_ctrl = round(results$percentExpr_ctrl, 6)

#Store results as a .csv file
write.csv(results, file = paste0('./', ctrl, '_vs_', case, '.csv'), row.names = F, col.names = T, quote = F)

[1] "NK CD56(dim)_Healthy" "NK CD56(dim)_COVID"  


R[write to console]: 6884shared cells



[1] "ctrl" "case"


DE Lists

In [34]:
# read tables and divide into up- and down-regulated genes for filtering
DE_table_RA = pd.read_csv('./NK CD56(dim)_Other_vs_NK CD56(dim)_RA.csv', index_col = 0)
DE_table_RA_sig_up = DE_table_RA[DE_table_RA['logFC'] > 0]
DE_table_RA_sig_down = DE_table_RA[DE_table_RA['logFC'] < 0]
DE_table_RA

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
AB_CD11b,-6.050228e-01,6.386196e-291,1.183809e-286,3.845620,4.450643,0.999435,0.999807
TXNIP,-8.537745e-01,2.149126e-203,1.991917e-199,1.829395,2.683170,0.854720,0.949797
AB_IgGFc,-5.947685e-01,4.849857e-196,2.996727e-192,3.247417,3.842186,0.998869,0.999421
AB_CD2,9.918110e-01,1.544014e-186,7.155345e-183,3.624814,2.633003,0.973997,0.924310
AB_CD45,5.517327e-01,1.573261e-164,5.832708e-161,3.495339,2.943607,0.993782,0.989573
...,...,...,...,...,...,...,...
CEP68,-1.928330e-06,9.996357e-01,9.998260e-01,0.020968,0.020970,0.020350,0.018923
MRPL44,-4.217503e-06,9.996768e-01,9.998260e-01,0.144801,0.144806,0.137366,0.133037
ERAP1,-3.742003e-06,9.997181e-01,9.998260e-01,0.146676,0.146680,0.139062,0.131493


In [35]:
DE_table_PS = pd.read_csv('./NK CD56(dim)_Other_vs_NK CD56(dim)_PS.csv', index_col = 0)
DE_table_PS_sig_up = DE_table_PS[DE_table_PS['logFC'] > 0]
DE_table_PS_sig_down = DE_table_PS[DE_table_PS['logFC'] < 0]
DE_table_PS

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
AB_CD99,-8.996892e-01,0.000000e+00,0.000000e+00,3.146553,4.046242,0.979845,1.000000
AB_CD47,-6.969145e-01,1.226373e-284,1.136664e-280,2.799824,3.496739,0.989922,0.999293
AB_TCRa/B,-7.652818e-01,3.118925e-272,1.927184e-268,2.418840,3.184122,0.986047,0.997879
AB_CD45RA,-8.229667e-01,6.635169e-260,3.074903e-256,4.713449,5.536415,0.999225,1.000000
AB_TCR,5.567678e-01,9.628503e-239,3.569671e-235,4.605209,4.048441,1.000000,0.999647
...,...,...,...,...,...,...,...
STAM-AS1,-2.495571e-06,9.990566e-01,9.992722e-01,0.004342,0.004344,0.003876,0.004242
NCK1,9.896183e-06,9.994199e-01,9.995817e-01,0.197461,0.197451,0.171318,0.177625
E2F8,1.387531e-06,9.995034e-01,9.996113e-01,0.003699,0.003697,0.003101,0.002828


In [36]:
DE_table_MS = pd.read_csv('./NK CD56(dim)_Other_vs_NK CD56(dim)_MS.csv', index_col = 0)
DE_table_MS_sig_up = DE_table_MS[DE_table_MS['logFC'] > 0]
DE_table_MS_sig_down = DE_table_MS[DE_table_MS['logFC'] < 0]
DE_table_MS

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
CX3CR1,7.696631e-01,0.000000e+00,0.000000e+00,0.945323,0.175660,0.625063,0.151221
AB_CD62P(P-Selectin),-1.303985e+00,0.000000e+00,0.000000e+00,2.086339,3.390324,0.934837,0.994549
XAF1,7.137907e-01,0.000000e+00,0.000000e+00,0.962077,0.248286,0.595489,0.211791
FCER1G,1.069771e+00,0.000000e+00,0.000000e+00,1.904013,0.834242,0.850125,0.485766
CXCR4,-9.809857e-01,2.150426e-284,7.972488e-281,0.809765,1.790750,0.478195,0.841308
...,...,...,...,...,...,...,...
USP48,7.380527e-06,9.994866e-01,9.996213e-01,0.188081,0.188074,0.167920,0.168181
ANP32E,-9.015551e-06,9.994996e-01,9.996213e-01,0.319128,0.319137,0.277193,0.272158
AC010175.1,-3.315463e-07,9.995669e-01,9.996213e-01,0.000545,0.000546,0.000501,0.000606


In [37]:
# here with respect to controls
DE_table_Control = pd.read_csv('./NK CD56(dim)_Control_vs_NK CD56(dim)_Diseased.csv', index_col = 0)
DE_table_Control_sig_up = DE_table_Control[DE_table_Control['logFC'] < 0]
DE_table_Control_sig_down = DE_table_Control[DE_table_Control['logFC'] > 0]
DE_table_Control

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
AB_IgM,-1.479977e+00,0.000000,0.000000,2.261228,3.741205,0.979620,1.000000
AB_IgA,-1.323790e+00,0.000000,0.000000,2.538875,3.862665,0.990898,1.000000
AB_CD235ab,-1.850082e+00,0.000000,0.000000,1.175392,3.025474,0.732291,0.989968
AB_IgGFc,-1.063513e+00,0.000000,0.000000,3.400844,4.464357,0.999011,1.000000
AB_Iglightchain-1,-1.074803e+00,0.000000,0.000000,3.011597,4.086401,0.998219,1.000000
...,...,...,...,...,...,...,...
C20orf197,1.191570e-06,0.999486,0.999618,0.004103,0.004102,0.003759,0.003696
DGKE,-4.285924e-06,0.999517,0.999618,0.064207,0.064211,0.059755,0.058606
TRIM36,-4.612535e-07,0.999564,0.999618,0.001042,0.001042,0.001187,0.001056


Sex filter

In [38]:
# filter genes differentially expressed either in the same direction for the females, RA, and MS comparisons, or for the males and control comparisons
DE_table_Ref = pd.read_csv('./NK CD56(dim)_Male_vs_Female.csv', index_col = 0)
DE_table_Ref_sig = DE_table_Ref[DE_table_Ref['adj.P.Val'] < 0.05]
DE_table_Ref_sig_up = DE_table_Ref_sig[DE_table_Ref_sig['logFC'] > 0]
DE_table_Ref_sig_down = DE_table_Ref_sig[DE_table_Ref_sig['logFC'] < 0]
DE_table_Ref

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
RPS4X,7.320186e-01,0.000000,0.000000,3.615830,2.883811,0.997561,0.976312
AREG,-1.089285e+00,0.000000,0.000000,0.150793,1.240078,0.088009,0.555394
HLA-C,-7.633274e-01,0.000000,0.000000,3.125446,3.888773,0.991931,0.998178
HLA-DRB1,9.788898e-01,0.000000,0.000000,1.303582,0.324692,0.656221,0.181487
CCL5,1.012443e+00,0.000000,0.000000,3.962843,2.950400,0.977669,0.872085
...,...,...,...,...,...,...,...
RPF1,9.158964e-06,0.999366,0.999599,0.176753,0.176744,0.121223,0.113703
TBC1D14,-7.082602e-06,0.999473,0.999647,0.150704,0.150711,0.103397,0.098032
SPACA6,1.874207e-06,0.999546,0.999663,0.012823,0.012822,0.008632,0.008382


In [39]:
list_genes_RA_up = []
curr_subset_DE = DE_table_RA_sig_up.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_RA_up.append(list(curr_subset_DE.index))
list_genes_RA_up = [item for sublist in list_genes_RA_up for item in sublist]
len(list_genes_RA_up)

7671

In [40]:
list_genes_RA_down = []
curr_subset_DE = DE_table_RA_sig_down.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_RA_down.append(list(curr_subset_DE.index))
list_genes_RA_down = [item for sublist in list_genes_RA_down for item in sublist]
len(list_genes_RA_down)

10866

In [41]:
list_genes_MS_up = []
curr_subset_DE = DE_table_MS_sig_up.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_MS_up.append(list(curr_subset_DE.index))
list_genes_MS_up = [item for sublist in list_genes_MS_up for item in sublist]
len(list_genes_MS_up)

11763

In [42]:
list_genes_MS_down = []
curr_subset_DE = DE_table_MS_sig_down.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_MS_down.append(list(curr_subset_DE.index))
list_genes_MS_down = [item for sublist in list_genes_MS_down for item in sublist]
len(list_genes_MS_down)

6774

In [43]:
list_genes_PS_up = []
curr_subset_DE = DE_table_PS_sig_up.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_PS_up.append(list(curr_subset_DE.index))
list_genes_PS_up = [item for sublist in list_genes_PS_up for item in sublist]
len(list_genes_PS_up)

7227

In [44]:
list_genes_PS_down = []
curr_subset_DE = DE_table_PS_sig_down.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_PS_down.append(list(curr_subset_DE.index))
list_genes_PS_down = [item for sublist in list_genes_PS_down for item in sublist]
len(list_genes_PS_down)

11310

In [45]:
list_genes_Control_up = []
curr_subset_DE = DE_table_Control_sig_up.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_Control_up.append(list(curr_subset_DE.index))
list_genes_Control_up = [item for sublist in list_genes_Control_up for item in sublist]
len(list_genes_Control_up)

4813

In [46]:
list_genes_Control_down = []
curr_subset_DE = DE_table_Control_sig_down.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_Control_down.append(list(curr_subset_DE.index))
list_genes_Control_down = [item for sublist in list_genes_Control_down for item in sublist]
len(list_genes_Control_down)

13724

In [47]:
list_genes_ref_up = []
curr_subset_DE = DE_table_Ref_sig_up.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_ref_up.append(list(curr_subset_DE.index))
list_genes_ref_up = [item for sublist in list_genes_ref_up for item in sublist]
len(list_genes_ref_up)

1411

In [48]:
list_genes_ref_down = []
curr_subset_DE = DE_table_Ref_sig_down.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_ref_down.append(list(curr_subset_DE.index))
list_genes_ref_down = [item for sublist in list_genes_ref_down for item in sublist]
len(list_genes_ref_down)

2270

In [49]:
genes_down_female = list(set(list_genes_ref_down) & set(list_genes_RA_down) & set(list_genes_MS_down))
len(genes_down_female)

159

In [50]:
genes_up_female = list(set(list_genes_ref_up) & set(list_genes_RA_up) & set(list_genes_MS_up))
len(genes_up_female)

323

In [51]:
genes_down_male = list(set(list_genes_ref_up) & set(list_genes_Control_down))
len(genes_down_male)

1011

In [52]:
genes_up_male = list(set(list_genes_ref_down) & set(list_genes_Control_up))
len(genes_up_male)

413

In [53]:
markers_RA_up = [ele for ele in list_genes_RA_up if ele not in genes_up_female]
markers_RA_down = [ele for ele in list_genes_RA_down if ele not in genes_down_female]
markers_RA = [markers_RA_up, markers_RA_down]
markers_RA = [item for sublist in markers_RA for item in sublist]
len(markers_RA)

18055

In [54]:
markers_MS_up = [ele for ele in list_genes_MS_up if ele not in genes_up_female]
markers_MS_down = [ele for ele in list_genes_MS_down if ele not in genes_down_female]
markers_MS = [markers_MS_up, markers_MS_down]
markers_MS = [item for sublist in markers_MS for item in sublist]
len(markers_MS)

18055

In [55]:
markers_Control_up = [ele for ele in list_genes_Control_up if ele not in genes_up_male]
markers_Control_down = [ele for ele in list_genes_Control_down if ele not in genes_down_male]
markers_Control = [markers_Control_up, markers_Control_down]
markers_Control = [item for sublist in markers_Control for item in sublist]
len(markers_Control)

17113

In [56]:
markers_PS = [list_genes_PS_up, list_genes_PS_down]
markers_PS = [item for sublist in markers_PS for item in sublist]
len(markers_PS)

18537

COVID filter

In [57]:
# subset genes to those differentially expressed in the COVID-19 vs. healthy comparison
DE_table_COVID = pd.read_csv('./NK CD56(dim)_Healthy_vs_NK CD56(dim)_COVID.csv', index_col = 0)
DE_table_COVID_sig = DE_table_COVID[DE_table_COVID['adj.P.Val'] < 0.05]
DE_table_COVID_sig

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
MTRNR2L8,1.132496,7.595587e-246,1.359610e-241,1.505164,0.372668,0.516138,0.205278
MT-CYB,-0.631185,3.592371e-228,3.215172e-224,2.926788,3.557973,0.964138,0.997238
EEF1A1,-0.314888,2.585867e-221,1.542900e-217,4.344396,4.659284,0.999724,1.000000
MT-CO3,-0.556180,5.807279e-215,2.598757e-211,3.088515,3.644695,0.982897,0.996625
HLA-DPB1,0.591804,4.287698e-197,1.534996e-193,0.971850,0.380047,0.593379,0.248236
...,...,...,...,...,...,...,...
CORO2A,0.009773,1.108312e-02,4.946095e-02,0.024926,0.015152,0.021241,0.011660
AC008966.1,0.005672,1.118312e-02,4.989477e-02,0.009007,0.003335,0.006897,0.002455
ABCD2,0.008436,1.120967e-02,4.998302e-02,0.018577,0.010141,0.015172,0.008285


In [58]:
DE_table_COVID_sig_up = DE_table_COVID_sig[DE_table_COVID_sig['logFC'] > 0]
DE_table_COVID_sig_down = DE_table_COVID_sig[DE_table_COVID_sig['logFC'] < 0]
len(DE_table_COVID_sig_up)

3339

In [59]:
len(DE_table_COVID_sig_down)

676

In [60]:
list_genes_COVID = []
curr_subset_DE = DE_table_COVID_sig.sort_values(by = 'adj.P.Val', ascending = True, axis = 0)
list_genes_COVID.append(list(curr_subset_DE.index))
list_genes_COVID = [item for sublist in list_genes_COVID for item in sublist]
len(list_genes_COVID)

4015

In [61]:
COVID_genes_RA = list(set(list_genes_COVID) & set(markers_RA))
len(COVID_genes_RA)

3738

In [62]:
COVID_genes_PS = list(set(list_genes_COVID) & set(markers_PS))
len(COVID_genes_PS)

4005

In [63]:
COVID_genes_MS = list(set(list_genes_COVID) & set(markers_MS))
len(COVID_genes_MS)

3738

In [64]:
COVID_genes_Ctrl = list(set(list_genes_COVID) & set(markers_Control))
len(COVID_genes_Ctrl)

3262

Export lists

In [65]:
# save filtered and unfiltered tables for genes and proteins
DE_table_RA_vs_rest_filtered1 = DE_table_RA[DE_table_RA.index.isin(COVID_genes_RA)]
DE_table_RA_vs_rest_filtered1.to_csv(r'/home/jovyan/COVID/NB7_TF/NK CD56(dim)_DE_table_RA_vs_rest_filtered_no_FDR.csv')
DE_table_RA_vs_rest_filtered2 = DE_table_RA_vs_rest_filtered1[DE_table_RA_vs_rest_filtered1['adj.P.Val'] < 0.05]
DE_table_RA_vs_rest_filtered2.to_csv(r'/home/jovyan/COVID/NB6_CellPhoneDB/DE_Tables/NK CD56(dim)_DE_table_RA_vs_rest_filtered.csv')
DE_table_RA_vs_rest_filtered1

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
CXCR4,7.163205e-01,4.628063e-135,7.799128e-132,2.043018,1.326698,0.884681,0.686619
CREM,3.833234e-01,6.705646e-105,6.215128e-102,0.658848,0.275525,0.434709,0.220506
HLA-DRB1,3.756096e-01,4.581733e-95,3.266599e-92,0.672785,0.297176,0.456190,0.229967
FCER1G,-5.749392e-01,1.027311e-87,6.801167e-85,0.712852,1.287792,0.421707,0.648002
OASL,2.345290e-01,1.192154e-68,4.804120e-66,0.391186,0.156656,0.322781,0.128403
...,...,...,...,...,...,...,...
F8,-9.514898e-06,9.969648e-01,9.982899e-01,0.007674,0.007683,0.007914,0.007337
MPP6,1.330114e-05,9.975378e-01,9.986152e-01,0.021962,0.021948,0.021481,0.019695
ZNF638,3.047141e-05,9.981750e-01,9.990373e-01,0.241250,0.241219,0.220464,0.208148


In [66]:
DE_table_PS_vs_rest_filtered1 = DE_table_PS[DE_table_PS.index.isin(COVID_genes_PS)]
DE_table_PS_vs_rest_filtered1.to_csv(r'/home/jovyan/COVID/NB7_TF/NK CD56(dim)_DE_table_PS_vs_rest_filtered_no_FDR.csv')
DE_table_PS_vs_rest_filtered2 = DE_table_PS_vs_rest_filtered1[DE_table_PS_vs_rest_filtered1['adj.P.Val'] < 0.05].copy()
DE_table_PS_vs_rest_filtered2.to_csv(r'/home/jovyan/COVID/NB6_CellPhoneDB/DE_Tables/NK CD56(dim)_DE_table_PS_vs_rest_filtered.csv')
DE_table_PS_vs_rest_filtered1

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
MT-CO1,0.530429,9.088048e-101,6.239450e-98,3.927353,3.396923,1.000000,0.996995
IFITM2,-0.625089,2.599286e-94,1.505718e-91,1.839997,2.465086,0.799225,0.928066
S100A8,0.434779,2.198802e-93,1.235127e-90,0.789919,0.355139,0.522481,0.267762
IFITM1,-0.606526,2.489758e-92,1.318647e-89,2.482334,3.088861,0.895349,0.953694
PRF1,-0.566111,3.953153e-69,1.332356e-66,1.681447,2.247558,0.765116,0.898551
...,...,...,...,...,...,...,...
BNIP2,-0.000070,9.969278e-01,9.983589e-01,0.414655,0.414725,0.340310,0.354719
PAFAH1B1,0.000071,9.969289e-01,9.983589e-01,0.410842,0.410771,0.334884,0.346589
SNF8,-0.000024,9.973549e-01,9.984860e-01,0.053768,0.053793,0.051163,0.051078


In [67]:
DE_table_MS_vs_rest_filtered1 = DE_table_MS[DE_table_MS.index.isin(COVID_genes_MS)]
DE_table_MS_vs_rest_filtered1.to_csv(r'/home/jovyan/COVID/NB7_TF/NK CD56(dim)_DE_table_MS_vs_rest_filtered_no_FDR.csv')
DE_table_MS_vs_rest_filtered2 = DE_table_MS_vs_rest_filtered1[DE_table_MS_vs_rest_filtered1['adj.P.Val'] < 0.05].copy()
DE_table_MS_vs_rest_filtered2.to_csv(r'/home/jovyan/COVID/NB6_CellPhoneDB/DE_Tables/NK CD56(dim)_DE_table_MS_vs_rest_filtered.csv')
DE_table_MS_vs_rest_filtered1

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
CX3CR1,0.769663,0.000000e+00,0.000000e+00,0.945323,0.175660,0.625063,0.151221
XAF1,0.713791,0.000000e+00,0.000000e+00,0.962077,0.248286,0.595489,0.211791
FCER1G,1.069771,0.000000e+00,0.000000e+00,1.904013,0.834242,0.850125,0.485766
CXCR4,-0.980986,2.150426e-284,7.972488e-281,0.809765,1.790750,0.478195,0.841308
CORO1A,0.801174,3.541870e-261,1.094261e-257,2.056371,1.255197,0.929323,0.750252
...,...,...,...,...,...,...,...
TRIM39,0.000014,9.973227e-01,9.985077e-01,0.021667,0.021653,0.020551,0.020594
MRPL50,0.000017,9.980195e-01,9.988817e-01,0.068360,0.068342,0.065163,0.064809
ADO,0.000003,9.991192e-01,9.994542e-01,0.011510,0.011507,0.010025,0.010701


In [68]:
DE_table_Control_vs_rest_filtered1 = DE_table_Control[DE_table_Control.index.isin(COVID_genes_Ctrl)]
DE_table_Control_vs_rest_filtered1.to_csv(r'/home/jovyan/COVID/NB7_TF/NK CD56(dim)_DE_table_Control_vs_rest_filtered_no_FDR.csv')
DE_table_Control_vs_rest_filtered2 = DE_table_Control_vs_rest_filtered1[DE_table_Control_vs_rest_filtered1['adj.P.Val'] < 0.05].copy()
DE_table_Control_vs_rest_filtered2.to_csv(r'/home/jovyan/COVID/NB6_CellPhoneDB/DE_Tables/NK CD56(dim)_DE_table_Control_vs_rest_filtered.csv')
DE_table_Control_vs_rest_filtered1

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
IGLV3-25,-0.329341,1.091939e-243,2.249031e-240,0.022620,0.351961,0.018401,0.269799
IGLV1-51,-0.198186,1.988791e-145,2.457749e-142,0.006185,0.204372,0.005342,0.151003
MT-CO1,0.549465,6.194110e-143,7.176264e-140,3.645188,3.095723,0.999209,0.993136
MT-ATP6,0.541811,1.266430e-122,1.304212e-119,2.300947,1.759136,0.962604,0.885428
IGHV5-51,-0.156075,3.941200e-115,3.320819e-112,0.009388,0.165463,0.007915,0.132524
...,...,...,...,...,...,...,...
CSRP1,-0.000168,9.858734e-01,9.911186e-01,0.125483,0.125651,0.119114,0.116156
SMIM12,-0.000157,9.879598e-01,9.925109e-01,0.154339,0.154497,0.145231,0.138860
BARD1,0.000031,9.959457e-01,9.978297e-01,0.047249,0.047218,0.043332,0.042767


In [69]:
DE_table_RA_protein = DE_table_RA[DE_table_RA.index.isin(protein_list)]
DE_table_RA_protein.to_csv('/home/jovyan/COVID/NB5_DE Analysis/DE proteins/NK CD56(dim)_DE_table_RA_vs_rest_protein.csv')
DE_table_RA_protein

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
AB_CD11b,-0.605023,6.386196e-291,1.183809e-286,3.845620,4.450643,0.999435,0.999807
AB_IgGFc,-0.594769,4.849857e-196,2.996727e-192,3.247417,3.842186,0.998869,0.999421
AB_CD2,0.991811,1.544014e-186,7.155345e-183,3.624814,2.633003,0.973997,0.924310
AB_CD45,0.551733,1.573261e-164,5.832708e-161,3.495339,2.943607,0.993782,0.989573
AB_TCRa/B,0.515205,8.362046e-150,2.583454e-146,3.426066,2.910861,1.000000,0.994207
...,...,...,...,...,...,...,...
AB_CD49b,0.008006,8.038292e-01,8.905972e-01,1.947126,1.939120,0.868287,0.838965
AB_Mouse_IgG2b,-0.000640,8.252205e-01,9.023767e-01,0.010074,0.010714,0.010741,0.010234
AB_CD158b(KIR2DL2),0.003851,8.788678e-01,9.340427e-01,2.914367,2.910515,0.987564,0.993242


In [70]:
DE_table_PS_protein = DE_table_PS[DE_table_PS.index.isin(protein_list)]
DE_table_PS_protein.to_csv('/home/jovyan/COVID/NB5_DE Analysis/DE proteins/NK CD56(dim)_DE_table_PS_vs_rest_protein.csv')
DE_table_PS_protein

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
AB_CD99,-0.899689,0.000000e+00,0.000000e+00,3.146553,4.046242,0.979845,1.000000
AB_CD47,-0.696915,1.226373e-284,1.136664e-280,2.799824,3.496739,0.989922,0.999293
AB_TCRa/B,-0.765282,3.118925e-272,1.927184e-268,2.418840,3.184122,0.986047,0.997879
AB_CD45RA,-0.822967,6.635169e-260,3.074903e-256,4.713449,5.536415,0.999225,1.000000
AB_TCR,0.556768,9.628503e-239,3.569671e-235,4.605209,4.048441,1.000000,0.999647
...,...,...,...,...,...,...,...
AB_CD71,-0.004517,8.707759e-01,9.308329e-01,1.401954,1.406471,0.817829,0.810887
AB_HLA-A2,0.001057,8.896420e-01,9.410976e-01,0.057584,0.056526,0.051163,0.053022
AB_mouse/ratCD278(ICOS),0.002633,9.208878e-01,9.582815e-01,2.088329,2.085696,0.959690,0.952633


In [71]:
DE_table_MS_protein = DE_table_MS[DE_table_MS.index.isin(protein_list)]
DE_table_MS_protein.to_csv('/home/jovyan/COVID/NB5_DE Analysis/DE proteins/NK CD56(dim)_DE_table_MS_vs_rest_protein.csv')
DE_table_MS_protein

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
AB_CD62P(P-Selectin),-1.303985,0.000000e+00,0.000000e+00,2.086339,3.390324,0.934837,0.994549
AB_CD235ab,-1.100213,3.843564e-252,1.017831e-248,0.895412,1.995625,0.623058,0.874823
AB_CD49b,-0.986591,1.749071e-239,3.602503e-236,1.237850,2.224441,0.674687,0.915607
AB_CD41,-1.380405,6.983420e-234,1.176833e-230,3.115313,4.495718,0.982957,0.986877
AB_CD15(SSEA-1),-0.850248,1.339410e-224,2.069054e-221,2.490928,3.341176,0.956391,0.994549
...,...,...,...,...,...,...,...
AB_CD44,0.009306,6.423507e-01,7.344717e-01,0.651334,0.642028,0.483208,0.458308
AB_HLA-A2,0.002844,6.641269e-01,7.519038e-01,0.058750,0.055906,0.053634,0.052292
AB_CD62L,0.009352,7.068128e-01,7.862995e-01,3.327438,3.318086,0.998496,0.992732


In [72]:
DE_table_Control_protein = DE_table_Control[DE_table_Control.index.isin(protein_list)]
DE_table_Control_protein.to_csv('/home/jovyan/COVID/NB5_DE Analysis/DE proteins/NK CD56(dim)_DE_table_Control_vs_rest_protein.csv')
DE_table_Control_protein

,logFC,P.Value,adj.P.Val,AveExpr_case,AveExpr_ctrl,percentExpr_case,percentExpr_ctrl
Gene,,,,,,,
AB_IgM,-1.479977,0.000000,0.000000,2.261228,3.741205,0.979620,1.000000
AB_IgA,-1.323790,0.000000,0.000000,2.538875,3.862665,0.990898,1.000000
AB_CD235ab,-1.850082,0.000000,0.000000,1.175392,3.025474,0.732291,0.989968
AB_IgGFc,-1.063513,0.000000,0.000000,3.400844,4.464357,0.999011,1.000000
AB_Iglightchain-1,-1.074803,0.000000,0.000000,3.011597,4.086401,0.998219,1.000000
...,...,...,...,...,...,...,...
AB_CD26,0.011153,0.621642,0.735848,1.517155,1.506002,0.852592,0.862724
AB_CD86,-0.008325,0.625702,0.739143,0.404401,0.412725,0.323704,0.327350
AB_CD64,0.003193,0.873727,0.920243,0.711348,0.708155,0.524733,0.518479


End of Notebook